# **Homework 1 - Text Mining**

## Group Members:
1. **Tarang Kadyan**  
   <tarang.kadyan@bse.eu>

2. **Oriol Gelabert**  
   <oriol.gelabert@bse.eu>

3. **Enzo Infantes**  
   <enzo.infantes@bse.eu>

<img src='https://upload.wikimedia.org/wikipedia/commons/4/41/BSE_primary_logo_color.jpg' width=300 />

# **1. Libraries**

In [45]:
import os
import time

import pandas as pd
import numpy as np
from packages.selenium_setup import *

## Preferences
This contains past preferences such as passwords, cookie acceptance etc.

Open Booking.com website to start our search:

In [2]:
dfolder= os.getcwd()
geko_path = os.path.join(dfolder, 'geckodriver.exe')
link='https://www.booking.com/index.es.html'


browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path)

Reject Cookies: The following cell closes Rejects the cookies, closing the banner and avoiding possible interference when scrapping.

In [3]:
path_cookies='//button[@id="onetrust-reject-all-handler"]'
cookies= browser.find_elements('xpath',path_cookies)
cookies[0].click()

Close Google Log-in

Google Log-In Pop-Up is inside an Iframe object, so we should first acces to the iframe and latter to the element we are willing to click, in this case the close button of this pop-up.

In [4]:
iframe_google = browser.find_elements(By.TAG_NAME,'iframe')[0] #we find the iframe object
browser.switch_to.frame(iframe_google) #switch to acces the iframe

In [5]:
close_log_in = browser.find_element(By.CSS_SELECTOR, '#close') # find the close button element
close_log_in.click() #click on it to close the pop-up
browser.switch_to.default_content() #switch back the browser to default to exit the iframe and continue web-scrapping

Find ''Where are you Going?'' Box and click on it:

In [6]:
browser.find_element(by=By.XPATH,value='//*[@id=":rh:"]').click()

We initially will search hotels in Barcelona, so we should input this value on the searchbox:

In [7]:
place = 'Barcelona'
search1 = browser.find_element(by=By.NAME,value='ss')
search1.send_keys(place)

Now, we select the dates of the event we wnat to track, in our case MWC. This event is hosted between 3-6 March, so our initial serach will be on the week of 1-8 March.

First we open the selector of dates:

In [8]:
css='button.ebbedaf8ac:nth-child(2) > span:nth-child(1)'

browser.find_element('css selector', css).click()

Advance to next month, to be able to select March dates:

In [9]:
path_next_month='//button[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 f671049264 f4552b6561 dc72a8413c f073249358"]'
next_month= browser.find_elements('xpath',path_next_month)
next_month[0].click()

We acces all dates that are now visible, including both February and March:

In [10]:
id = "calendar-searchboxdatepicker"
span_class = "cf06f772fa ef091eb985"
path = f'//div[@id="{id}"]//table[@class="eb03f3f27f"]//tbody//td[@class="b80d5adb18"]//span[@class="{span_class}"]'
dates = browser.find_elements('xpath',path)
for date in dates:
    print(date.get_attribute("data-date"))

2025-03-01
2025-03-02
2025-03-03
2025-03-04
2025-03-05
2025-03-06
2025-03-07
2025-03-08
2025-03-09
2025-03-10
2025-03-11
2025-03-12
2025-03-13
2025-03-14
2025-03-15
2025-03-16
2025-03-17
2025-03-18
2025-03-19
2025-03-20
2025-03-21
2025-03-22
2025-03-23
2025-03-24
2025-03-25
2025-03-26
2025-03-27
2025-03-28
2025-03-29
2025-03-30
2025-03-31
2025-04-01
2025-04-02
2025-04-03
2025-04-04
2025-04-05
2025-04-06
2025-04-07
2025-04-08
2025-04-09
2025-04-10
2025-04-11
2025-04-12
2025-04-13
2025-04-14
2025-04-15
2025-04-16
2025-04-17
2025-04-18
2025-04-19
2025-04-20
2025-04-21
2025-04-22
2025-04-23
2025-04-24
2025-04-25
2025-04-26
2025-04-27
2025-04-28
2025-04-29
2025-04-30


In [11]:
from_day = input('Day from which you want to look for (In march)') #important two digits
to_day = input('Day until which you want to look for (In march)')


for date in dates:
    if date.get_attribute("data-date") == f"2025-03-{from_day}":
        date.click()
    if date.get_attribute("data-date") == f"2025-03-{to_day}":
        date.click()
        break

Bedroom selection

In [12]:
people_selection_path='//button[@class="a83ed08757 ebbedaf8ac dbaff8df6f ada2387af8"]'
people_selection= browser.find_elements('xpath',people_selection_path)
people_selection[0].click()

ONE ADULT MORE

In [13]:
path_cookies='//button[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 ab98298258 bb803d8689 f4d78af12a"]'
cookies= browser.find_elements('xpath',path_cookies)
cookies[0].click()

ONE ADULT LESS

In [14]:
path_cookies='//button[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 ab98298258 bb803d8689 e91c91fa93"]'
cookies= browser.find_elements('xpath',path_cookies)
cookies[0].click()

In [15]:
my_xpath="/html/body/div[3]/div[2]/div/form/div/div[4]/button/span"

check_obscures(browser, my_xpath , type='xpath')
check_and_click(browser,my_xpath , type='xpath')

Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div/div[4]/button/span; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div/div[4]/button/span; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.

In [16]:
# Geniuos rejected
path_cookies='//div[@class="abcc616ec7 cc1b961f14 c180176d40 f11eccb5e8 ff74db973c"]'
cookies= browser.find_elements('xpath',path_cookies)
cookies[0].click()

In [ ]:
def scroll_and_click():
    while True:
        try:
            #scroll to the end of the page
            path_final_results = '//div[@class="extranet_link_container"]'
            final_results = browser.find_elements(By.XPATH, path_final_results)
            browser.execute_script("arguments[0].scrollIntoView();", final_results[0])
            time.sleep(2) #wait for the page to load

            #click on the button to load more results
            path_total_results = '//button[@class="a83ed08757 c21c56c305 bf0537ecb5 f671049264 af7297d90d c0e0affd09"]'    
            button = browser.find_element(By.XPATH, path_total_results)
            button.click()
            time.sleep(2) #wait for the page to load
        except NoSuchElementException:
            print("There aren't more results to load")
            break

scroll_and_click()

There aren't more results to load


In [30]:
# Extracting the most relevant information about the hotels
rating = '//div[@class="a3b8729ab1 d86cee9b25"]'
name = '//div[@class="f6431b446c a15b38c233"]'
price = '//span[@class="f6431b446c fbfd7c1165 e84eb96b1f"]'
location = '//span[@class="aee5343fdb def9bc142a" and @data-testid="address"]'


ratings = browser.find_elements('xpath', rating)
names = browser.find_elements('xpath', name)
prices = browser.find_elements('xpath', price)
locations = browser.find_elements('xpath', location)

In [ ]:
# Extracting the links for all the hotels
link_elements = browser.find_elements(By.CSS_SELECTOR, 'a[data-testid="title-link"]')
links = [element.get_attribute("href") for element in link_elements]

In [51]:
# all the information is stored in a dataframe
data = []
for i in range(len(name)):
    data.append({
        'name': names[i].text, 
        'rating': ratings[i].text, 
        'price': prices[i].text, 
        'location': locations[i].text, 
        'link': links[i]
    })

df = pd.DataFrame(data, columns=['name', 'rating', 'price', 'location', 'link'])

In [65]:
df.head()

,name,rating,price,location,link
0,Tembo Barcelona,"Puntuación: 8,8\n8,8",€ 222,Barcelona,https://www.booking.com/hotel/es/tembo-barcelo...
1,Le Meridien Barcelona,"Puntuación: 7,6\n7,6",€ 298,"Ciutat Vella, Barcelona",https://www.booking.com/hotel/es/le-meridien-b...
2,Hotel Lleó,"Puntuación: 8,3\n8,3",€ 97,"Ciutat Vella, Barcelona",https://www.booking.com/hotel/es/lleo.es.html?...
3,Hotel Lloret Ramblas,"Puntuación: 8,1\n8,1",€ 76,"Ciutat Vella, Barcelona",https://www.booking.com/hotel/es/lloret.es.htm...
4,Hotel Market,"Puntuación: 7,6\n7,6",€ 103,"Eixample, Barcelona",https://www.booking.com/hotel/es/market.es.htm...


In [63]:
pd.reset_option('display.max_colwidth')